In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import contextily as ctx
import statsmodels.api as sm

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, precision_recall_fscore_support
from sklearn.feature_selection import RFE

In [2]:
parcels = pd.read_csv('./data/Blue Sky Code and Inputs/SF_Logistic_Data.csv')

In [3]:
allParcels = gpd.read_file('./data/Parcels   Active and Retired/parcels.shp')

In [4]:
sites = gpd.read_file('../housing-elements/data/raw_data/housing_sites/xn--Bay_Area_Housing_Opportunity_Sites_Inventory__20072023_-it38a.shp')

In [5]:
tax = pd.read_excel('./data/tax_assessor/2019.8.20__SF_ASR_Secured_Roll_Data_2007-2008.xlsx')

In [6]:
use_codes = pd.read_excel('./data/tax_assessor/2019.8.20__SF_ASR_Secured_Roll_Data_2007-2008.xlsx', 
                          sheet_name='Class & Use Code')

In [7]:
neighborhood_codes = pd.read_excel('./data/tax_assessor/2019.8.20__SF_ASR_Secured_Roll_Data_2007-2008.xlsx', 
                                   sheet_name='Neigborhood Code')

### Training Set is RHNA 4

In [8]:
trainParcels = parcels[np.logical_and(parcels.year >= 2007, parcels.year <= 2015)]

In [9]:
trainY = trainParcels.groupby('MapBlkLot_Master')['Developed'].agg(lambda x: x.ne(0).sum())

In [10]:
trainY.sum()

308

In [11]:
round(trainY.mean(), 3) * 100

0.2

In [12]:
trainX = trainParcels[trainParcels.year == 2007]

No duplicative index.

In [13]:
nunique_lots = trainParcels[trainParcels.year == 2007].MapBlkLot_Master.nunique()
n_lots = trainParcels[trainParcels.year == 2007].shape[0]
assert nunique_lots == n_lots

In [14]:
trainX.MapBlkLot_Master.isin(trainY.index).mean()

1.0

In [15]:
trainDf = pd.merge(trainX.drop('Developed', axis=1), trainY, left_on='MapBlkLot_Master', right_index=True)

In [16]:
trainDf.Developed.value_counts()

0    152910
1       306
2         1
Name: Developed, dtype: int64

In [17]:
def clean_apn(apn):
    apn = ''.join(apn.split(' '))
    if len(apn) < 9:
        return apn
    block_length = 4
    #if apn[block_length].isalpha():
    #    return apn[:block_length] + apn[block_length+1:]
    return apn
 
    
tax['MapBlkLot_Master'] = tax.RP1PRCLID.apply(clean_apn)

In [18]:
trainDf.MapBlkLot_Master.isin(tax.MapBlkLot_Master).mean().round(2)

0.98

### Get 2% of parcels that are in BlueSky for 2008 but not in tax data

I have 3514 parcels to identify. 

In [19]:
cantID = trainDf[~trainDf.MapBlkLot_Master.isin(tax.MapBlkLot_Master)]

In [20]:
cantID.shape

(3514, 17)

Only 170 of x total missing developed parcels are missing because I cannot identify them at this stage.

In [21]:
cantID.Developed.sum()

170

I look at all active and retired parcels to find their geometry.

In [22]:
allParcels = allParcels.dissolve(by='mapblklot')

In [23]:
allParcels = allParcels[['geometry']]

In [24]:
cantID = allParcels.merge(cantID,
                          how='inner',
                          right_on='MapBlkLot_Master',
                          left_on='mapblklot',
                          validate='one_to_one')

In [25]:
cantID.shape

(3513, 18)

So now I have geometries for all but one 3514 parcels. Now I have to find geometries for tax data. Unfortunately, after above dissolve, only 78% have a match. It seems that the BlueSky and Tax datasets may disagree on whether a condo complex deserves an ID or a condo unit deserves an ID, I think. I assume tax assessor needs per-unit data

In [26]:
tax.MapBlkLot_Master.isin(allParcels.index).mean().round(2)

0.78

In [27]:
tax.shape

(197778, 33)

In [28]:
taxGeo = allParcels.merge(tax,
                          how='inner',
                          right_on='MapBlkLot_Master',
                          left_on='mapblklot',
                          validate='one_to_one')

In [29]:
taxGeo.shape

(154008, 34)

In [30]:
taxGeo.crs == cantID.crs

True

In [31]:
canID = gpd.sjoin(cantID, taxGeo, how="inner", predicate='within')

In [32]:
canID.shape

(2632, 52)

I should check the above result is the same if I just do a merge on the geometry since that requires an exact match, which is what I'm assuming the above code does, but I'm not certain.

Okay, so there were 170 developed parcels I could not identify in the tax data. And 92 of them are recoverable with the above geospatial join approach. That leaves 78 still missing. I'm getting a bit more than half

I can probably find the other five by using Reza's permit id approach as a Plan A, Reza's mapblklot approach as Plan B, and geospatial as a Plan C. 

In [33]:
canID.Developed.sum()

92

In [34]:
170 - 92

78

In [35]:
round((canID.shape[0] / cantID.shape[0]), 2)

0.75

In [36]:
round((canID.Developed.sum() / cantID.Developed.sum()), 2)

0.54

So I get 75% of the missing parcels recovered. This is not much worse than the percentage of tax parcels not in the allParcels dataframe so I suspect using the geospatial version of tax will solve this nearly entirely.

Only recover 54% of the developed parcels. 

In [37]:
canID.columns

Index(['geometry', 'MapBlkLot_Master_left', 'year', 'Historic',
       'Residential_Dummy', 'Zillow_Price_Real', 'Const_FedReserve_Real',
       'Envelope_1000', 'Upzone_Ratio', 'zp_OfficeComm', 'zp_DensRestMulti',
       'zp_FormBasedMulti', 'zp_PDRInd', 'zp_Public', 'zp_Redev', 'zp_RH2',
       'zp_RH3_RM1', 'Developed', 'index_right', 'PROPLOC', 'RP1NBRCDE',
       'RP1PRCLID', 'RP1VOLUME', 'RP1CLACDE', 'YRBLT', 'BATHS', 'BEDS',
       'ROOMS', 'STOREYNO', 'UNITS', 'ZONE', 'CONSTTYPE', 'DEPTH', 'FRONT',
       'SQFT', 'FBA', 'LAREA', 'LOTCODE', 'REPRISDATE', 'RP1TRACDE',
       'OWNRPRCNT', 'EXEMPTYPE', 'RP1STACDE', 'RP1EXMVL2', 'RP1EXMVL1',
       'ROLLYEAR', 'RECURRSALD', 'RP1FXTVAL', 'RP1IMPVAL', 'RP1LNDVAL',
       'RP1PPTVAL', 'MapBlkLot_Master_right'],
      dtype='object')

The following demonstrates how the mapblklot's are different even though the geometries are the same. If I cared more, I would dig into the Active and Retired Parcels dataset to figure out the story here.

In [38]:
canID[['MapBlkLot_Master_left', 'MapBlkLot_Master_right']].sample(10, random_state=0)

,MapBlkLot_Master_left,MapBlkLot_Master_right
138,0196032,0196029
613,0922044,0922023
1712,3549094,3549018C
748,1080051,1080024
1343,1933061,1933024
2370,4591C151,4591A076
2264,4273034,4273005
2046,3773010,3773300A
3075,6597071,6597031
3081,6600054,6600012A


In [39]:
canID['MapBlkLot_Master'] = canID['MapBlkLot_Master_left']

In [40]:
canID = canID.drop(['MapBlkLot_Master_left', 'MapBlkLot_Master_right', 'geometry', 'index_right'], axis=1)

### Merge tax data and parcel data

In [41]:
trainDf.shape

(153217, 17)

In [42]:
trainDf.Developed.sum()

308

In [43]:
trainDf = trainDf.merge(tax, how='inner', on='MapBlkLot_Master')

Merging with tax data loses most of the developed parcels because I'm using inner. I already knew the nrows of cantid was 170 parcels, so this is what I expect.

In [44]:
trainDf.Developed.sum()

138

Concatenate the geomatched parcels back in.

In [45]:
trainDf.MapBlkLot_Master.nunique() == trainDf.shape[0]

True

In [46]:
trainDf.MapBlkLot_Master.isin(canID.MapBlkLot_Master).sum()

0

#### TODO: This concatenation introduces two non-unique mapblklot identifiers because canID is returning multiple rows per parcel.

In [47]:
trainDf = pd.concat((trainDf, canID), axis=0)

In [48]:
trainDf.Developed.sum()

230

In [58]:
trainDf.index[trainDf.index.duplicated()]

Index(['3533175', '6551060'], dtype='object', name='MapBlkLot_Master')

#### Fix the two observations below. Some funny business is happening with spatial join to get two observations for these bad boys.

In [64]:
canID[canID.MapBlkLot_Master == '6551060']

,year,Historic,Residential_Dummy,Zillow_Price_Real,Const_FedReserve_Real,Envelope_1000,Upzone_Ratio,zp_OfficeComm,zp_DensRestMulti,zp_FormBasedMulti,...,RP1STACDE,RP1EXMVL2,RP1EXMVL1,ROLLYEAR,RECURRSALD,RP1FXTVAL,RP1IMPVAL,RP1LNDVAL,RP1PPTVAL,MapBlkLot_Master
3032,2007,0,1,93.227099,92.120253,2.720602,2.0,0,0,0,...,NaN,0,0,7,50715,0,646680,970020,0,6551060
3032,2007,0,1,93.227099,92.120253,2.720602,2.0,0,0,0,...,NaN,0,0,7,0,0,0,0,0,6551060


In [68]:
canID[canID.MapBlkLot_Master == '3533175']

,year,Historic,Residential_Dummy,Zillow_Price_Real,Const_FedReserve_Real,Envelope_1000,Upzone_Ratio,zp_OfficeComm,zp_DensRestMulti,zp_FormBasedMulti,...,RP1STACDE,RP1EXMVL2,RP1EXMVL1,ROLLYEAR,RECURRSALD,RP1FXTVAL,RP1IMPVAL,RP1LNDVAL,RP1PPTVAL,MapBlkLot_Master
1656,2007,0,1,93.227099,92.120253,8.012521,2.0,0,1,0,...,NaN,0,0,7,40430,0,0,1750993,0,3533175
1656,2007,0,1,93.227099,92.120253,8.012521,2.0,0,1,0,...,NaN,0,0,7,0,0,0,0,0,3533175


In [69]:
trainDf.MapBlkLot_Master.nunique() - trainDf.shape[0]

AttributeError: 'DataFrame' object has no attribute 'MapBlkLot_Master'

So, as expected, I'm short 78 developed parcels.

In [52]:
trainDf = trainDf.set_index('MapBlkLot_Master')

In [53]:
trainDf.columns

Index(['year', 'Historic', 'Residential_Dummy', 'Zillow_Price_Real',
       'Const_FedReserve_Real', 'Envelope_1000', 'Upzone_Ratio',
       'zp_OfficeComm', 'zp_DensRestMulti', 'zp_FormBasedMulti', 'zp_PDRInd',
       'zp_Public', 'zp_Redev', 'zp_RH2', 'zp_RH3_RM1', 'Developed', 'PROPLOC',
       'RP1NBRCDE', 'RP1PRCLID', 'RP1VOLUME', 'RP1CLACDE', 'YRBLT', 'BATHS',
       'BEDS', 'ROOMS', 'STOREYNO', 'UNITS', 'ZONE', 'CONSTTYPE', 'DEPTH',
       'FRONT', 'SQFT', 'FBA', 'LAREA', 'LOTCODE', 'REPRISDATE', 'RP1TRACDE',
       'OWNRPRCNT', 'EXEMPTYPE', 'RP1STACDE', 'RP1EXMVL2', 'RP1EXMVL1',
       'ROLLYEAR', 'RECURRSALD', 'RP1FXTVAL', 'RP1IMPVAL', 'RP1LNDVAL',
       'RP1PPTVAL'],
      dtype='object')

In [54]:
trainDf.shape

(152335, 48)

Somewhere above, I've introduced two non-unique indexes.

In [55]:
trainX = trainDf.drop(['year', 'Developed', 'RP1PRCLID', 'PROPLOC', 
                      'Const_FedReserve_Real', 'Zillow_Price_Real', 'zp_RH2'], axis=1)

In [56]:
trainX.shape

(152335, 41)

In [78]:
trainX = trainX.loc[~trainX.index.duplicated()]

In [79]:
trainX.shape

(152333, 41)

In [82]:
trainY = trainY[~trainY.index.duplicated()]

In [83]:
trainY.shape

(152333,)

In [84]:
trainX.describe()

,Historic,Residential_Dummy,Envelope_1000,Upzone_Ratio,zp_OfficeComm,zp_DensRestMulti,zp_FormBasedMulti,zp_PDRInd,zp_Public,zp_Redev,...,RP1TRACDE,OWNRPRCNT,RP1EXMVL2,RP1EXMVL1,ROLLYEAR,RECURRSALD,RP1FXTVAL,RP1IMPVAL,RP1LNDVAL,RP1PPTVAL
count,152333.000000,152333.000000,152333.000000,152333.000000,152333.000000,152333.000000,152333.000000,152333.000000,152333.000000,152333.000000,...,152247.000000,152333.000000,1.523330e+05,152333.000000,152333.0,152333.000000,1.523330e+05,1.523330e+05,1.523330e+05,1.523330e+05
mean,0.134016,0.911070,4.118889,0.772937,0.012131,0.100280,0.004917,0.018512,0.010044,0.002133,...,1000.519091,0.885773,2.733110e+04,3547.215639,7.0,245533.642973,6.020360e+03,3.442140e+05,3.283077e+05,4.048780e+03
std,0.340670,0.284644,23.828903,0.660828,0.109472,0.300374,0.069948,0.134794,0.099714,0.046141,...,20.678732,0.227196,1.326790e+06,4338.142932,0.0,403072.734091,8.103218e+05,3.471683e+06,1.748172e+06,2.854760e+05
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1000.000000,0.000000,0.000000e+00,0.000000,7.0,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000,1.000000,1.000000,0.443853,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1000.000000,1.000000,0.000000e+00,0.000000,7.0,0.000000,0.000000e+00,6.033600e+04,4.085800e+04,0.000000e+00
50%,0.000000,1.000000,1.000000,0.561167,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1000.000000,1.000000,0.000000e+00,0.000000,7.0,0.000000,0.000000e+00,1.433740e+05,1.433960e+05,0.000000e+00
75%,0.000000,1.000000,2.000000,0.952381,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1000.000000,1.000000,0.000000e+00,7000.000000,7.0,870318.000000,0.000000e+00,2.801290e+05,3.374060e+05,0.000000e+00
max,1.000000,1.000000,2297.421872,34.800000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,9000.000000,1.000000,3.763004e+08,777000.000000,7.0,991231.000000,3.017421e+08,6.120000e+08,2.560200e+08,5.345948e+07


In [85]:
trainX.iloc[:,5:].describe()

,zp_DensRestMulti,zp_FormBasedMulti,zp_PDRInd,zp_Public,zp_Redev,zp_RH3_RM1,RP1VOLUME,YRBLT,BATHS,BEDS,...,RP1TRACDE,OWNRPRCNT,RP1EXMVL2,RP1EXMVL1,ROLLYEAR,RECURRSALD,RP1FXTVAL,RP1IMPVAL,RP1LNDVAL,RP1PPTVAL
count,152333.000000,152333.000000,152333.000000,152333.000000,152333.000000,152333.000000,152333.000000,152333.000000,152333.000000,152333.000000,...,152247.000000,152333.000000,1.523330e+05,152333.000000,152333.0,152333.000000,1.523330e+05,1.523330e+05,1.523330e+05,1.523330e+05
mean,0.100280,0.004917,0.018512,0.010044,0.002133,0.126440,22.492402,1859.286950,2.723350,0.872011,...,1000.519091,0.885773,2.733110e+04,3547.215639,7.0,245533.642973,6.020360e+03,3.442140e+05,3.283077e+05,4.048780e+03
std,0.300374,0.069948,0.134794,0.099714,0.046141,0.332346,12.485101,371.914773,9.708732,11.971218,...,20.678732,0.227196,1.326790e+06,4338.142932,0.0,403072.734091,8.103218e+05,3.471683e+06,1.748172e+06,2.854760e+05
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,1000.000000,0.000000,0.000000e+00,0.000000,7.0,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,12.000000,1909.000000,1.000000,0.000000,...,1000.000000,1.000000,0.000000e+00,0.000000,7.0,0.000000,0.000000e+00,6.033600e+04,4.085800e+04,0.000000e+00
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,21.000000,1926.000000,2.000000,0.000000,...,1000.000000,1.000000,0.000000e+00,0.000000,7.0,0.000000,0.000000e+00,1.433740e+05,1.433960e+05,0.000000e+00
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,35.000000,1947.000000,3.000000,0.000000,...,1000.000000,1.000000,0.000000e+00,7000.000000,7.0,870318.000000,0.000000e+00,2.801290e+05,3.374060e+05,0.000000e+00
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,44.000000,8687.000000,850.000000,3800.000000,...,9000.000000,1.000000,3.763004e+08,777000.000000,7.0,991231.000000,3.017421e+08,6.120000e+08,2.560200e+08,5.345948e+07


In [86]:
(trainX.YRBLT[trainX.YRBLT> 2008]).max()

8687

In [136]:
trainY.sum()

229

### Feature Engineering

#### Use Codes

In [89]:
use_codes = use_codes[~use_codes.isna().all(axis=1)]

In [90]:
use_codes.head()

,USE,DESC,CLASS,DESC.1
0,SRES,Single Family Residential,CO,Coop Units Unsegregated
1,SRES,Single Family Residential,COS,Coop Units Segregated
2,SRES,Single Family Residential,D,Dwelling
3,SRES,Single Family Residential,DBM,Dwelling BMR
4,SRES,Single Family Residential,LZ,Live/Work Condominium


In [91]:
use_codes = use_codes[use_codes['CLASS'] != 'CLASS']
use_lookup = use_codes.groupby('CLASS')['USE'].agg(list).to_dict()
use_lookup = {k: v[-1] for k, v in use_lookup.items()}

In [92]:
trainX['general_use_code'] = trainX.RP1CLACDE.map(use_lookup)

#### New features

In [93]:
trainX['hasBMR'] = trainX.RP1CLACDE.str.endswith('BM')

#### Neighborhood Code

In [94]:
neighborhood_codes.NEIGHBORHOOD = neighborhood_codes.NEIGHBORHOOD.str.strip().str.lower().str.split(' ').str.join('_')

In [95]:
neighborhood_codes.head()

,DISTRICT,CODE,NEIGHBORHOOD,BOUNDRIES
0,1,1A,central_richmond,"South of California, Park Presidio, south of F..."
1,1,1B,inner_richmond,"South of California, Arguello, south of Fulton..."
2,1,1C,jordan_park/laurel_heights,"California, west of Presidio, Geary, Arguello"
3,1,1D,lake_--the_presidio,"West and south of Presidio Terrace, Arguello, ..."
4,1,1E,outer_richmond,"The Ocean, west of 32nd Avenue, south of Fulton"


In [96]:
neighborhoods = {k: v[0] for k, v in neighborhood_codes.groupby('CODE')['NEIGHBORHOOD'].agg(list).to_dict().items()}
neighborhoods = {(k if len(k) == 3 else '0'+k): v for k, v in neighborhoods.items()} 

In [97]:
def lookup_neighborhood(x):
    return neighborhoods.get(x, x)

In [98]:
trainX['neighborhood'] = trainX.RP1NBRCDE.apply(lookup_neighborhood)

In [99]:
districts = {k: v[0] for k, v in neighborhood_codes.groupby('CODE')['DISTRICT'].agg(list).to_dict().items()}
districts = {(k if len(k) == 3 else '0'+k): v for k, v in districts.items()} 

In [100]:
def lookup_district(x):
    return 'district' + str(districts.get(x, x))

In [101]:
trainX['district'] = trainX['RP1NBRCDE'].apply(lookup_district)

In [102]:
trainX = trainX.drop('RP1NBRCDE', axis=1)

### Drop constant columns

In [103]:
trainDf.shape[0] 

152335

In [104]:
trainX = trainX.drop(trainX.columns[trainX.nunique() <= 1], axis=1)

In [105]:
trainX.shape

(152333, 42)

### Baseline model is human model

Evaluate precision, recall, etc.

In [106]:
rhna4Sites = sites[sites['rhnacyc'] == 'RHNA4']
rhna4Sites = rhna4Sites[rhna4Sites.jurisdict == 'San Francisco']

In [107]:
rhna4Sites.shape

(4604, 35)

In [108]:
rhna4Sites.locapn = rhna4Sites.locapn.str.split('/').str.join('')

In [109]:
rhna4Sites.apn = rhna4Sites.apn.str.split('/').str.join('')

In [110]:
round(rhna4Sites.locapn.isin(trainY.index).mean(), 2)

0.85

In [111]:
round(rhna4Sites.apn.isin(trainY.index).mean(), 2)

0.95

In [112]:
# TODO: Fix this APN matching
rhna4Sites = rhna4Sites[np.logical_or(rhna4Sites.locapn.isin(trainY.index), 
                                      rhna4Sites.apn.isin(trainY.index))]

In [113]:
rhna4Sites.shape

(4386, 35)

In [114]:
predictedSites = trainY.index.isin(rhna4Sites.locapn.values)

In [115]:
predictedSites.size

152333

The site inventory has a 1.5% chance of development.

In [116]:
round(trainY[predictedSites].mean() * 100, 1)

1.4

The parcels not in the site inventory has a 0.2% chance of development.

In [117]:
predicted = pd.Series([1]*trainY[predictedSites].size + [0]*trainY[~predictedSites].size, 
                      index=trainY.index[predictedSites].tolist() + trainY.index[~predictedSites].tolist())

In [118]:
f1_score(predicted, trainY>0)

0.0004861448711716092

In [119]:
precision, recall, fscore, support = precision_recall_fscore_support(predicted, trainY>0)

In [120]:
precision

array([0.97446484, 0.00436681])

### Treat inclusion in site inventory as a feature

In [122]:
trainX = trainX[~trainX.index.duplicated(keep='first')]

In [123]:
predicted = predicted[~predicted.index.duplicated(keep='first')]

In [124]:
trainX['inInventory'] = predicted

In [125]:
trainY = trainY[~trainY.index.duplicated(keep='first')]

#### Save trainX and trainY to csv for R

In [126]:
trainX.to_csv('./data/clean_data/trainX.csv')

In [127]:
trainY.to_csv('./data/clean_data/trainY.csv')

### Dummies

In [129]:
trainX.columns

Index(['Historic', 'Residential_Dummy', 'Envelope_1000', 'Upzone_Ratio',
       'zp_OfficeComm', 'zp_DensRestMulti', 'zp_FormBasedMulti', 'zp_PDRInd',
       'zp_Public', 'zp_Redev', 'zp_RH3_RM1', 'RP1VOLUME', 'RP1CLACDE',
       'YRBLT', 'BATHS', 'BEDS', 'ROOMS', 'STOREYNO', 'UNITS', 'ZONE',
       'CONSTTYPE', 'DEPTH', 'FRONT', 'SQFT', 'FBA', 'LAREA', 'LOTCODE',
       'RP1TRACDE', 'OWNRPRCNT', 'EXEMPTYPE', 'RP1STACDE', 'RP1EXMVL2',
       'RP1EXMVL1', 'RECURRSALD', 'RP1FXTVAL', 'RP1IMPVAL', 'RP1LNDVAL',
       'RP1PPTVAL', 'general_use_code', 'hasBMR', 'neighborhood', 'district',
       'inInventory'],
      dtype='object')

In [130]:
trainX = pd.get_dummies(trainX, drop_first=True)

In [131]:
trainX.shape

(152333, 368)

### Drop rows that have nans

I have to do this after getting dummies bc a lot of NANs drop out naturally from getting dummies.

In [132]:
nonNanX, nonNany = trainX[~trainX.isna().any(axis=1)], trainY[~trainX.isna().any(axis=1)]

In [133]:
nonNanX.shape

(152247, 368)

In [134]:
nonNany = (nonNany > 0).astype(int)

In [135]:
nonNany.sum()

222

### Identify multicollinearity issues

Also have to do this after getting dummies because a lot of the neighborhood and district codes are highly correlated.

In [137]:
corrdf = trainX.corr()

In [138]:
corrdf = round(corrdf, 2)

In [139]:
upper = corrdf.where(np.triu(np.ones(corrdf.shape), k=1).astype(bool))

# Find features with correlation greater than 0.95
to_drop = {column:sum(upper[column] > 0.95) for column in upper.columns if any(upper[column] > 0.95)}

In [140]:
list(to_drop.keys())

['general_use_code_COMO',
 'general_use_code_COMR',
 'general_use_code_IND',
 'district_district08I',
 'district_district09B']

In [141]:
len(to_drop)

5

In [142]:
trainX = trainX.drop(to_drop, axis=1)

### Am I missing any permitted parcels?

So I am missing 79 developed parcels

In [143]:
n_start = parcels[np.logical_and(parcels.year >= 2007, parcels.year <= 2015)].Developed.sum()
n_end = nonNany.sum()
n_start - n_end

86

In [144]:
nonNany.shape

(152247,)

In [145]:
n_start

308

That's 28% attrition

In [146]:
round((n_start - n_end) / n_start, 2)

0.28

Whereas I only lose 1% of parcels overall.

In [147]:
n_start = parcels[parcels.year == 2007].shape[0]
n_end = trainY.shape[0]
round((n_start - n_end) / n_start, 2)

0.01

Thus, missing observations are not missing at random.

### PCA

In [ ]:
from sklearn import decomposition
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(nonNanX)
X_train_std = sc.transform(nonNanX)

pca = decomposition.PCA()
pca.fit(X_train_std)
X = pca.transform(X_train_std)

In [ ]:
X_train_pca = pca.fit_transform(X_train_std)
#
# Determine explained variance using explained_variance_ration_ attribute
#
exp_var_pca = pca.explained_variance_ratio_
#
# Cumulative sum of eigenvalues; This will be used to create step plot
# for visualizing the variance explained by each principal component.
#
cum_sum_eigenvalues = np.cumsum(exp_var_pca)
#
# Create the visualization plot
#
plt.bar(range(0,len(exp_var_pca)), exp_var_pca, alpha=0.5, align='center', label='Individual explained variance')
plt.step(range(0,len(cum_sum_eigenvalues)), cum_sum_eigenvalues, where='mid',label='Cumulative explained variance')
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal component index')
plt.legend(loc='best')
plt.tight_layout()
plt.show()

### Fit models to RHNA5. See what we learn

Is there non iid behavior in the data? This score seems way too high. So maybe parcels that are nearby are predicted separately and that makes this easier?

In [ ]:
estimator =  LogisticRegression(C=1, penalty='l1', solver='liblinear')
selector = RFE(estimator = estimator, n_features_to_select = 25, step=50)

In [ ]:
selector = selector.fit(nonNanX, nonNany)

In [ ]:
keepCols = nonNanX.columns[selector.get_support()]

In [ ]:
keepCols

### Does logistic regression identify selection of site inventory as important?

In [ ]:
trainX2 = nonNanX[keepCols].copy()

In [ ]:
trainX2['inInventory'].mean()

In [ ]:
trainX2.sort_index(inplace=True)

In [ ]:
nonNany.sort_index(inplace=True)

In [ ]:
trainX2 = sm.add_constant(trainX2.astype(float))

In [ ]:
lm = sm.Logit(nonNany, trainX2)

In [ ]:
result = lm.fit(solver='bfgs')

In [ ]:
result.summary()

### Test Set is RHNA 5

In [ ]:
0.9979963058929492